# Merging Datasets

In [12]:
import pandas as pd
import os
import sys
from tqdm import tqdm
tqdm.pandas()
data_directory = "data/"


## Paired Datasets

In [ ]:

print(os.path.abspath(data_directory))



aiSource = pd.read_csv(data_directory + "ai-code/output.csv")

C:\Desktop 2\Repos\Code-Classifier\data


In [78]:
print(aiSource)


    identifier                                               code actual label
0   Z5-Z3-4415  \n#include <stdio.h>\n#include <stdlib.h>\n#in...           ai
1   Z5-Z3-1132  \n#include <stdio.h>\n#include <time.h>\n\ntyp...           ai
2   Z5-Z2-8794  \n#include <stdio.h>\n#include <stdlib.h>\n\ni...           ai
3   Z5-Z2-3315  \n#include <stdio.h>\n\nint main() {\n    char...           ai
4   Z5-Z2-3186  \n#include <stdio.h>\n#include <stdlib.h>\n\n#...           ai
5   Z5-Z1-7485  \n#include <stdio.h>\n#include <stdlib.h>\n#in...           ai
6   Z5-Z1-5298  \n#include <stdio.h>\n#include <stdlib.h>\n\n/...           ai
7   Z5-Z1-3258  \n#include <stdio.h>\n\nint main() {\n    int ...      ai-0.8t
8   Z4-Z4-9318  \n#include <string>\nusing namespace std;\n\ni...           ai
9   Z4-Z4-8774  \n#include <stdio.h>\n#include <stdlib.h>\n\nv...           ai
10  Z4-Z4-7392  \n#include <vector>\n#include <string>\n#inclu...           ai
11  Z4-Z3-8794  \n#include <stdio.h>\n#include <stdl

In [79]:
print(aiSource['identifier'])

0     Z5-Z3-4415
1     Z5-Z3-1132
2     Z5-Z2-8794
3     Z5-Z2-3315
4     Z5-Z2-3186
5     Z5-Z1-7485
6     Z5-Z1-5298
7     Z5-Z1-3258
8     Z4-Z4-9318
9     Z4-Z4-8774
10    Z4-Z4-7392
11    Z4-Z3-8794
12    Z4-Z3-8532
13    Z4-Z3-4766
14    Z4-Z2-9188
15    Z4-Z2-5284
16    Z4-Z2-2675
17    Z4-Z1-7421
18    Z4-Z1-6956
19    Z4-Z1-5185
20    Z3-Z3-6151
21    Z3-Z3-5612
22    Z3-Z3-2806
23    Z3-Z2-9935
24    Z3-Z2-8480
25    Z3-Z2-3756
26    Z3-Z1-8670
27    Z3-Z1-7597
28    Z3-Z1-1477
29    Z2-Z4-8923
30    Z2-Z4-2925
31    Z2-Z4-2086
32    Z2-Z3-4192
33    Z2-Z3-2574
34    Z2-Z3-2197
35    Z2-Z2-8532
36    Z2-Z2-5488
37    Z2-Z2-3736
38    Z2-Z1-8085
39    Z2-Z1-6054
40    Z2-Z1-3517
41    Z1-Z4-9296
42    Z1-Z4-7013
43    Z1-Z4-4682
44    Z1-Z3-9923
45    Z1-Z3-7755
46    Z1-Z3-5581
47    Z1-Z2-9998
48    Z1-Z2-4016
49    Z1-Z2-1772
50    Z1-Z1-8085
51    Z1-Z1-4647
52    Z1-Z1-1066
Name: identifier, dtype: object


In [80]:
dataset_directory = "datasets/VedranLjubovic/chosen/"
humanCode = aiSource['identifier'].to_frame()
humanCode['code'] = [None for x in range(len(humanCode))]

def getCode(fileIdentifier):
  try:
    file = open(dataset_directory + fileIdentifier + ".c", "r")
    code = "\n".join(file.readlines())
    file.close()
  except Exception as e:
    sys.stderr.write("Error reading file: " + dataset_directory + fileIdentifier + ".c" + "\n")
    return None
  return code

humanCode['code'] = humanCode['identifier'].progress_apply(getCode)

print(humanCode)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 53/53 [00:00<00:00, 1664.66it/s]

    identifier                                               code
0   Z5-Z3-4415  #include <stdio.h>\n\n#include <string.h>\n\n#...
1   Z5-Z3-1132  #include <stdio.h>\n\n#include <string.h>\n\n#...
2   Z5-Z2-8794  #include <stdio.h>\n\n#include <math.h>\n\n#in...
3   Z5-Z2-3315  #include <stdio.h>\n\n#include <ctype.h>\n\n#i...
4   Z5-Z2-3186  #include <stdio.h>\n\n#include <stdlib.h>\n\n#...
5   Z5-Z1-7485  #include <stdio.h>\n\n#include <string.h>\n\n#...
6   Z5-Z1-5298  #include <stdio.h>\n\n#include <string.h>\n\n\...
7   Z5-Z1-3258  #include <stdio.h>\n\n\n\nstruct Student {\n\n...
8   Z4-Z4-9318  #include <stdio.h>\n\n#include <math.h>\n\n#de...
9   Z4-Z4-8774  #include <stdio.h>\n\n\n\nvoid dbr(char *p,cha...
10  Z4-Z4-7392  #include <stdio.h>\n\n#include <math.h>\n\n\n\...
11  Z4-Z3-8794  #include <stdio.h>\n\n\n\n//Zadaca 4 by @White...
12  Z4-Z3-8532  #include <stdio.h>\n\n\n\nchar* zamjena_rijeci...
13  Z4-Z3-4766  #include <stdio.h>\n\n\n\nchar *zamjena_rijeci...
14  Z4-Z2-

In [81]:
human_df = humanCode['code'].copy()
ai_df = aiSource['code'].copy()

human_df.columns = ['code']
ai_df.columns = ['code']

human_df = human_df.to_frame()
ai_df = ai_df.to_frame()

human_df['actual label'] = "human"
ai_df['actual label'] = "llm"

merged_df = pd.concat([human_df, ai_df], ignore_index=True)

In [82]:
merged_df.to_pickle(data_directory + "prepared/" + "code.pkl")

# splitting data

In [2]:
embedding_directory = "data/prepared/run2/"
ratios = [80, 10, 10]

data = pd.read_pickle(embedding_directory + "embeddings.pkl")
data_num = len(data)

train_split = int(ratios[0]/sum(ratios)*data_num)
val_split = train_split + int(ratios[1]/sum(ratios)*data_num)

data = data.sample(frac=1, random_state=666)
train = data.iloc[:train_split]
dev = data.iloc[train_split:val_split]
test = data.iloc[val_split:]

print(train_split, val_split)

84 94


In [3]:
train.to_pickle(embedding_directory + "train.pkl")
dev.to_pickle(embedding_directory + "dev.pkl")
test.to_pickle(embedding_directory + "test.pkl")

In [11]:
print(train)

   actual label                                    code_embeddings
8         human  [-0.0055792592, 0.06428501, -0.036523633, -0.0...
57          llm  [-0.027133392, 0.009359762, -0.10520409, 0.011...
90          llm  [0.015876377, -0.053451587, -0.070226625, 0.01...
40        human  [-0.045699924, 0.02933702, -0.11228899, 0.0292...
67          llm  [0.008493862, 0.032250702, -0.08035441, -0.015...
..          ...                                                ...
11        human  [0.021025768, 0.010113711, -0.13329825, 0.0339...
20        human  [0.0009624716, 0.020897575, -0.10187468, 0.063...
86          llm  [-0.06238021, 0.041018948, -0.10947401, 0.0711...
13        human  [0.02036846, -0.012620583, -0.12502861, -0.029...
79          llm  [-0.060971193, 0.070538305, -0.16201116, -0.00...

[84 rows x 2 columns]


In [26]:
# filtered_data = train.loc[:, train.columns.str.startswith('code_')]
# filtered_data['actual label'] = train['actual label']
# filtered_data['code_embeddings'] = filtered_data['code_embeddings'].apply(lambda x: x.flatten())


data_expanded = pd.DataFrame(train['code_embeddings'].tolist(), index=train.index)  # Expands each ndarray into separate columns
filtered_data = pd.concat([data_expanded, train[['actual label']]], axis=1)
# print(train['actual label'])
print(filtered_data) 

           0         1         2         3         4         5         6  \
8  -0.005579  0.064285 -0.036524 -0.022370  0.007151 -0.011820  0.000123   
57 -0.027133  0.009360 -0.105204  0.011347 -0.132737  0.003184 -0.019521   
90  0.015876 -0.053452 -0.070227  0.016451 -0.106307 -0.039220 -0.111423   
40 -0.045700  0.029337 -0.112289  0.029299 -0.094496  0.048511 -0.039841   
67  0.008494  0.032251 -0.080354 -0.015844 -0.059486  0.064062 -0.086975   
..       ...       ...       ...       ...       ...       ...       ...   
11  0.021026  0.010114 -0.133298  0.033915 -0.011136 -0.005381 -0.018329   
20  0.000962  0.020898 -0.101875  0.063812 -0.064141 -0.003297  0.016214   
86 -0.062380  0.041019 -0.109474  0.071166 -0.063021  0.049664 -0.035394   
13  0.020368 -0.012621 -0.125029 -0.029516 -0.077637 -0.018517 -0.033507   
79 -0.060971  0.070538 -0.162011 -0.003407 -0.026897  0.116769 -0.018240   

           7         8         9  ...       247       248       249       250  \
8   0.

# Different Testing Dataset

In [3]:
testDirectory = "datasets/VedranLjubovic/testChosen/"
testFiles = os.listdir(testDirectory)

In [14]:
humanTestCode = pd.DataFrame()
humanTestCode['code'] = [x for x in testFiles if getCode(x) is not None]

def getCode(fileIdentifier):
  try:
    file = open(testDirectory + fileIdentifier, "r")
    code = "\n".join(file.readlines())
    file.close()
  except Exception as e:
    sys.stderr.write("Error reading file: " + testDirectory + fileIdentifier + ".c" + "\n")
    return None
  return code

humanTestCode['code'] = humanTestCode['code'].progress_apply(getCode)

print(humanTestCode)

Error reading file: datasets/VedranLjubovic/testChosen/Z3-Z2-4770.c.c
Error reading file: datasets/VedranLjubovic/testChosen/Z4-Z2-2160.c.c
Error reading file: datasets/VedranLjubovic/testChosen/Z4-Z4-2254.c.c
Error reading file: datasets/VedranLjubovic/testChosen/Z4-Z4-7125.c.c
Error reading file: datasets/VedranLjubovic/testChosen/Z5-Z1-7888.c.c
100%|███████████████████████████████████████████████████████| 175/175 [00:00<00:00, 8822.68it/s]

                                                  code
0    #include <stdio.h>\n\n\n\nint main() {\n\n\tdo...
1    #include <stdio.h>\n\n\n\nint main() {\n\n\tfl...
2    #include <stdio.h>\n\n\n\n#define I_parcijalni...
3    #include <stdio.h>\n\n#define eps 0.0001\n\nin...
4    #include <stdio.h>\n\n\n\nint main() {\n\n\tin...
..                                                 ...
170  #include <stdio.h>\n\n#include <string.h>\n\n#...
171  /*\n\n#include <stdio.h>\n\n#include <string.h...
172  #include <stdio.h>\n\n#include <string.h>\n\n#...
173  #include <stdio.h>\n\n#include <string.h>\n\n#...
174  #include <stdio.h>\n\n#include <string.h>\n\n#...

[175 rows x 1 columns]


In [15]:
humanTestCode['actual label'] = "human"

In [10]:
print(humanTestCode)

                                                  code actual label
0    #include <stdio.h>\n\n\n\nint main() {\n\n\tdo...        human
1    #include <stdio.h>\n\n\n\nint main() {\n\n\tfl...        human
2    #include <stdio.h>\n\n\n\n#define I_parcijalni...        human
3    #include <stdio.h>\n\n#define eps 0.0001\n\nin...        human
4    #include <stdio.h>\n\n\n\nint main() {\n\n\tin...        human
..                                                 ...          ...
175  #include <stdio.h>\n\n#include <string.h>\n\n#...        human
176  /*\n\n#include <stdio.h>\n\n#include <string.h...        human
177  #include <stdio.h>\n\n#include <string.h>\n\n#...        human
178  #include <stdio.h>\n\n#include <string.h>\n\n#...        human
179  #include <stdio.h>\n\n#include <string.h>\n\n#...        human

[180 rows x 2 columns]


In [16]:
humanTestCode.to_pickle(data_directory + "prepared/" + "testCode.pkl")